In [49]:
# Class: COEN 140 Lab Machine Learning/Data Mining
# Name: Matthew Davenport
# Date: 9/25/2022
# Title: Lab 5 – Crime predictions using linear and ridge regression without python libraries
# Description: This program will perform clssifications
#           on crime datasets
#           without using python ML libraries
#           implementing linear and ridge regression

import numpy as np
import math
import time

In [50]:
# import data from txt file for test data and training data
import urllib.request
import pandas as pd

test_data = urllib.request.urlopen('https://www.cse.scu.edu/~yfang/coen140/crime-test.txt')
train_data = urllib.request.urlopen('https://www.cse.scu.edu/~yfang/coen140/crime-train.txt')

train_data = pd.read_csv(train_data, delimiter='\t')
test_data = pd.read_csv(test_data, delimiter='\t')
print(train_data)
print(test_data)


      ViolentCrimesPerPop  population  householdsize  agePct12t21  \
0                    0.67       -0.45          -1.85        -1.06   
1                    0.43       -0.45          -0.27        -0.22   
2                    0.12       -0.14           1.87         0.55   
3                    0.03       -0.38           0.53        -0.28   
4                    0.14       -0.30          -1.12        -0.74   
...                   ...         ...            ...          ...   
1590                 0.09       -0.38          -0.39         0.04   
1591                 0.45       -0.06           3.03         1.71   
1592                 0.23        0.81          -0.57        -0.48   
1593                 0.19        0.18           0.28         1.00   
1594                 0.48        1.12           1.93         0.49   

      agePct12t29  agePct16t24  agePct65up  numbUrban  pctUrban  medIncome  \
0            0.67         0.08       -0.85      -0.34      0.68      -0.24   
1           -0.

In [51]:
# format data (separations for X and y, and adding dummy feature)
y_test = test_data.iloc[:, 0]
y_train = train_data.iloc[:, 0]
X_test = test_data.iloc[: , 1:]
X_train = train_data.iloc[: , 1:]

# correct the type to float64
X_train = np.float64(X_train)
X_test = np.float64(X_test)
y_train = np.float64(y_train)
y_test = np.float64(y_test)

# add dummy feature to X training and testing matrix
X_train = np.column_stack((X_train, np.ones(len(X_train))))
X_test = np.column_stack((X_test, np.ones(len(X_test))))


# df = df.iloc[: , 1:] dataframe keeping everything except first column 
# dataframe.iloc[:, 0] dataframe extracting only first column

print(X_test)
print(y_test)

[[-0.14  0.35 -0.41 ...  0.65 -0.39  1.  ]
 [ 0.02 -0.45 -0.22 ... -0.66 -0.39  1.  ]
 [-0.45  0.28 -0.16 ... -0.66 -0.39  1.  ]
 ...
 [-0.38  1.99  1.07 ... -0.57 -0.39  1.  ]
 [-0.38  0.04 -0.22 ... -0.27 -0.39  1.  ]
 [ 0.02 -0.57 -0.48 ... -0.14 -0.39  1.  ]]
[0.08 0.22 0.06 0.16 0.15 0.28 0.63 0.16 0.25 1.   0.04 0.05 0.86 0.08
 0.09 0.17 0.12 0.12 1.   0.16 0.4  0.12 0.07 0.4  0.06 0.21 0.04 0.45
 0.02 0.28 0.38 0.22 0.07 0.06 0.16 0.21 1.   0.19 0.26 0.1  0.23 0.09
 0.55 1.   0.09 0.41 1.   0.05 0.02 0.09 0.11 0.36 0.03 0.63 0.04 0.02
 0.08 0.27 0.52 0.27 0.31 0.06 0.1  0.22 0.6  0.49 0.4  0.02 0.21 0.2
 1.   0.5  0.15 0.31 0.22 0.16 0.16 0.32 0.21 0.09 0.09 0.02 0.05 0.15
 0.11 0.1  0.06 0.11 0.01 0.1  0.18 0.85 0.05 0.12 0.54 0.06 0.04 0.18
 0.44 0.03 0.14 0.03 0.29 0.14 0.71 0.06 0.35 0.07 0.2  0.3  0.05 0.05
 0.26 0.62 0.11 0.03 0.17 0.25 0.34 0.39 0.13 0.19 0.11 0.06 0.08 0.01
 0.04 0.25 0.08 0.31 1.   0.73 0.18 0.37 1.   0.37 0.04 0.05 0.04 0.2
 0.33 0.06 0.03 0.29 1.   0.

In [52]:
def RMSE(predicted_outputs, correct_outputs):
    total = 0
    differences = []
    m = len(predicted_outputs)
    differences = (predicted_outputs - correct_outputs) ** 2
    for difference in differences: 
        total += difference
    error = math.sqrt(total/m)
    return error

Part 1: Implement ridge regression model with lambda = 400 -> lamba = 0.78 in steps of /2. Compute the RMSE value on the training data and test data, respectively. Report both RSME values. Provide a function that, given an NxP numpy array of data points, returns an Nx1 array of predicted outputs. Determine the best value for lambda from those used.

In [53]:
def problem1(X, y, validate, reg_lambda):
    # reg_lambda = 100
    predicted_outputs = []
    w = np.dot((np.linalg.inv((np.dot(X.T, X)) + reg_lambda * np.identity(len(X.T)))), \
               np.dot(X.T, y))
    for x in validate:
        predicted_outputs.append(np.dot(x.T, w))
        
    return predicted_outputs

Now apply ridge regression to the test and train data sets 10 times with RMSE values to determine best lambda. 

In [54]:
reg_lambda = 400
RMSE_values = np.zeros((10,5))

for i in range(10):
    for k in range(5):
        # first split dataset into training and validation sets
        # for both X and y.. splutting using k = 5, 4/5 for training, 1/5 for testing
        # Training sets: 
        X_training = np.concatenate((X_train[0:int(k*(len(X_train)/5))], X_train[int((k+1)*(len(X_train)/5)):int(len(X_train))]))
        y_training = np.concatenate((y_train[0:int(k*(len(y_train)/5))], y_train[int((k+1)*(len(y_train)/5)):int(len(y_train))]))
        # Validation sets: 
        X_validation = X_train[int(k*(len(X_train)/5)):int((k+1)*(len(X_train)/5))]
        y_validation = y_train[int(k*(len(y_train)/5)):int((k+1)*(len(y_train)/5))]
        
        # compute predictions for training and testing data
        ridge_pred = problem1(X_training, y_training, X_validation, reg_lambda)
        # compute RMSE for the predictions using current k and lambda value
        RMSE_values[i, k] = RMSE(ridge_pred, y_validation)

        # display results
        print("RMSE for ridge iteration   ", i + 1, "with k value ", k + 1, ": ", RMSE_values[i,k])
        print("Lambda value for iteration ", i + 1, "with k value ", k + 1, ": ", reg_lambda, "\n\n")

    # divide lambda by 2 for next iteration
    reg_lambda /= 2 


RMSE for ridge iteration    1 with k value  1 :  0.1603162601526477
Lambda value for iteration  1 with k value  1 :  400 


RMSE for ridge iteration    1 with k value  2 :  0.14530190822461106
Lambda value for iteration  1 with k value  2 :  400 


RMSE for ridge iteration    1 with k value  3 :  0.15755165412304029
Lambda value for iteration  1 with k value  3 :  400 


RMSE for ridge iteration    1 with k value  4 :  0.1466920217861653
Lambda value for iteration  1 with k value  4 :  400 


RMSE for ridge iteration    1 with k value  5 :  0.13770203666732556
Lambda value for iteration  1 with k value  5 :  400 


RMSE for ridge iteration    2 with k value  1 :  0.15049469712623617
Lambda value for iteration  2 with k value  1 :  200.0 


RMSE for ridge iteration    2 with k value  2 :  0.1391640252519248
Lambda value for iteration  2 with k value  2 :  200.0 


RMSE for ridge iteration    2 with k value  3 :  0.14576514733030577
Lambda value for iteration  2 with k value  3 :  200.0 

Results:

For the testing iteration, the lowest RMSE value was produced by using lambda = 12.5 on iteration 6. 

In [55]:
print(RMSE_values)

[[0.16031626 0.14530191 0.15755165 0.14669202 0.13770204]
 [0.1504947  0.13916403 0.14576515 0.13872299 0.12932324]
 [0.14641669 0.13761947 0.13961026 0.13589421 0.12684322]
 [0.14511564 0.13746282 0.1363316  0.13498737 0.1268823 ]
 [0.1449868  0.13758002 0.13455271 0.13464334 0.12781642]
 [0.14528919 0.13780086 0.13365482 0.13444721 0.12891979]
 [0.14570088 0.13815894 0.13329282 0.13430567 0.12988101]
 [0.14610098 0.13865628 0.13324682 0.13421881 0.1306319 ]
 [0.1464384  0.13923262 0.13336903 0.13420359 0.13119456]
 [0.14668497 0.13978827 0.13355288 0.13425675 0.13159847]]


In [67]:
avg = 0
totals = []
for i in range(9): 
    for k in range(4):
        avg += RMSE_values[i, k] 
    avg /= 5
    totals.append(avg)
    avg = 0
    
print(totals)
minimum = min(totals)
print(minimum)
idx = totals.index(minimum)
print(idx)
optimal_lambda = 400
for i in range(idx - 1):
    optimal_lambda /= 2
print(optimal_lambda)


[0.12197236885729286, 0.1148293711253671, 0.11190812780068668, 0.11077948350099046, 0.11035257454666543, 0.11023841718121292, 0.11029166254690333, 0.11044457760809949, 0.11064872815578157]
0.11023841718121292
5
25.0


Problem 2: Using Linear Regresion with gradient descent. For the initial weights, you can just use Gaussian N(0, 1) random variables. Define “converging” as the change in any coefficient between one iteration and the next is no larger than a small value (e.g., .

In [68]:
#initial w0 value using Gaussian random variables as described in lab doc:
w0 = np.random.normal(0,1,(len(X_train.T),))
l1 = 1
l0 = 0
alpha = 5 * (10**(-5))

# w^(t+1) = w^t - alpha * dl(w^t)/dw^t
# convergence achieved once change in w is less than 10^-5
while (abs(l1 - l0) > (10**-5)):
    w1 = w0 - (alpha * np.dot(X_train.T, (np.dot(X_train, w0) - y_train)))
    l0 = np.dot(((y_train - np.dot(X_train,w0)).T), (y_train - np.dot(X_train,w0)))
    l1 = np.dot(((y_train - np.dot(X_train,w1)).T), (y_train - np.dot(X_train,w1)))
    w0 = w1

In [69]:
# return NX1 array of predictions using w0 obtained with gradient descenet and linear regression
def problem2(samples):
    predicted_outputs = []
    for X in samples: 
        predicted_outputs.append(np.dot(X.T, w0))
    return predicted_outputs 

predictions = problem2(X_train)
test = problem2(X_test)

pred_RMSE = RMSE(predictions, y_train)
test_RMSE = RMSE(test, y_test)

print("RMSE for training: ",pred_RMSE)
print("RMSE for testing:  ", test_RMSE)

RMSE for training:  0.12789740016039447
RMSE for testing:   0.14609210135101197


In [70]:
def problem3(samples, w): 
    predicted_outputs = []
    for X in samples: 
        predicted_outputs.append(np.dot(X.T, w))
    return predicted_outputs

In [71]:
RMSE_GD_values = []
for k in range(5):
    # first split dataset into training and validation sets
    # for both X and y.. splutting using k = 5, 4/5 for training, 1/5 for testing
    # Training sets: 
    X_training = np.concatenate((X_train[0:int(k*(len(X_train)/5))], X_train[int((k+1)*(len(X_train)/5)):int(len(X_train))]))
    y_training = np.concatenate((y_train[0:int(k*(len(y_train)/5))], y_train[int((k+1)*(len(y_train)/5)):int(len(y_train))]))
    # Validation sets: 
    X_validation = X_train[int(k*(len(X_train)/5)):int((k+1)*(len(X_train)/5))]
    y_validation = y_train[int(k*(len(y_train)/5)):int((k+1)*(len(y_train)/5))]
        
    #initial w0 value using Gaussian random variables as described in lab doc:
    w0 = np.random.normal(0,1,(len(X_train.T),))
    l1 = 1
    l0 = 0
    alpha = 5 * (10**(-5))

    # w^(t+1) = w^t - alpha * dl(w^t)/dw^t
    # convergence achieved once change in w is less than 10^-5
    while (abs(l1 - l0) > (10**-5)):
        w1 = w0 - (alpha * np.dot(X_train.T, (np.dot(X_train, w0) - y_train)))
        l0 = np.dot(((y_training - np.dot(X_training, w0)).T), (y_training - np.dot(X_training, w0)))
        l1 = np.dot(((y_training - np.dot(X_training, w1)).T), (y_training - np.dot(X_training, w1)))
        w0 = w1
    RMSE_GD_values.append(RMSE(problem3(X_validation, w0), y_validation))
RMSE_training = min(RMSE_GD_values)

    

In [72]:
#initial w0 value using Gaussian random variables as described in lab doc:
w0 = np.random.normal(0,1,(len(X_train.T),))
l1 = 1
l0 = 0
alpha = 5 * (10**(-5))

# w^(t+1) = w^t - alpha * dl(w^t)/dw^t
# convergence achieved once change in w is less than 10^-5
while (abs(l1 - l0) > (10**-5)):
    w1 = w0 - (alpha * np.dot(X_train.T, (np.dot(X_train, w0) - y_train)))
    l0 = np.dot(((y_train - np.dot(X_train,w0)).T), (y_train - np.dot(X_train,w0)))
    l1 = np.dot(((y_train - np.dot(X_train,w1)).T), (y_train - np.dot(X_train,w1)))
    w0 = w1
    
RMSE_testing = RMSE(problem3(X_test, w0), y_test)
print(RMSE_testing)

0.1459220030429634


Final Results for part 3: 

In [73]:
print("RMSE for training using ridge regression and gradient descent: ",RMSE_training)
print("RMSE for testing using ridge regression and gradient descent:  ",RMSE_testing)

RMSE for training using ridge regression and gradient descent:  0.12168089149950062
RMSE for testing using ridge regression and gradient descent:   0.1459220030429634
